<a href="https://colab.research.google.com/github/Priyo-prog/Time-series-analysis/blob/main/Business%20Case/volkswagen_buyout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Business Case : Volkswagen buyout of Porsche**

## Pip Install

In [3]:
!pip3 install numpy scipy patsy pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip3 install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
!pip install arch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 903 kB 4.3 MB/s 
     |████████████████████████████████| 9.8 MB 35.8 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [6]:
!pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 4.1 MB/s 


Delete the "!pip uninstall statsmodels" once the installation is complete

## Import the Packages and Libraries

In [7]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from pmdarima.arima import OCSBTest
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import yfinance # yahoo finanace
import warnings
warnings.filterwarnings("ignore")
sns.set()

## Importing the Data

In [8]:
raw_data = yfinance.download(tickers="VOW3.DE, PAH3.DE, BMW.DE", interval="1d", group_by="ticker",
                             auto_adjust=True, treads=True)

[*********************100%***********************]  3 of 3 completed


In [9]:
df = raw_data.copy()

## Defining Key Dates

In [10]:
## Starting Date
start_date = '2009-04-05'

## First Official Announcement - 49.9%
ann_1 = '2009-12-09'

## Second Official Announcement - 50.1%
ann_2 = '2012-07-05'

## Ending Date
end_date = '2014-01-01'

## Dieselgate scanndal
d_gate = '2015-09-20'

## Pre-processing the Data

In [13]:
## Extracting Closing Prices
df['vol'] = df['VOW3.DE'].Close
df['por'] = df['PAH3.DE'].Close
df['bmw'] = df['BMW.DE'].Close

## Creating Returns (We'll be examimng volatility)
df['ret_vol'] = df['vol'].pct_change(1).mul(100)
df['ret_por'] = df['por'].pct_change(1).mul(100)
df['ret_bmw'] = df['bmw'].pct_change(1).mul(100)

## Creating Squared Returns
df['sq_vol'] = df.ret_vol.mul(df.ret_vol)
df['sq_por'] = df.ret_por.mul(df.ret_por)
df['sq_bmw'] = df.ret_bmw.mul(df.ret_bmw)

## Extracting Volume (Quantity extracted on that day)
df['q_vol'] = df['VOW3.DE'].Volume
df['q_por'] = df['PAH3.DE'].Volume
df['q_bmw'] = df['BMW.DE'].Volume

In [15]:
## Assigning the frequency and Filling NA Values
df = df.asfreq('b')
df = df.fillna(method='bfill')

In [16]:
## Removing Surplus Data
del df['VOW3.DE'], df['PAH3.DE'], df['BMW.DE']

In [17]:
df.head()

,vol,por,bmw,ret_vol,ret_por,ret_bmw,sq_vol,sq_por,sq_bmw,q_vol,q_por,q_bmw
,,,,,,,,,,,,
Date,,,,,,,,,,,,
1996-11-08,32.022877,25.596838,9.333746,-4.405639,-3.831346,-0.269660,19.409656,14.67921,0.072716,56336.0,901566.0,767000.0
1996-11-11,32.022877,25.596838,9.308577,-4.405639,-3.831346,-0.269660,19.409656,14.67921,0.072716,56336.0,901566.0,260000.0
1996-11-12,32.022877,25.596838,9.378950,-4.405639,-3.831346,0.756007,19.409656,14.67921,0.571547,56336.0,901566.0,1066000.0
1996-11-13,32.022877,25.596838,9.364054,-4.405639,-3.831346,-0.158828,19.409656,14.67921,0.025226,56336.0,901566.0,793000.0
1996-11-14,32.022877,25.596838,9.394359,-4.405639,-3.831346,0.323630,19.409656,14.67921,0.104737,56336.0,901566.0,351000.0
